# Mobiliar Challenge

See: https://mobiliar.jacando.jobs/ch/de/job/eriaH43V?layout=partners_iframe_header

## Problem Statement
The following string is given:

In [1]:
mystery = '''aHR0cHM6Ly9jb3MtY2h hbGxlbmdlLnMzLmV1LW NlbnRyYWwtMS5hbWF6b

25hd3MuY29tL2RkMGUw

YTM2ODUyNGI0MjUzYzV lZjNiMzMwZWJlNjcxYT VlNjVmYTY0YzQyZWI2M

DA3Yzk5YWNjMjczNmJk

Nwo='''

## Solution

required python modules:

In [2]:
import base64
import urllib2
import hexdump
from capstone import *
from unicorn import *
from unicorn.x86_const import *

First, decode the base64 string:

In [3]:
url = base64.b64decode(mystery).decode('utf-8').strip()
print(url)

https://cos-challenge.s3.eu-central-1.amazonaws.com/dd0e0a368524b4253c5ef3b330ebe671a5e65fa64c42eb6007c99acc2736bd7


The result is a file hosted on a [Amazon Simple Storage Service (S3)](https://aws.amazon.com/de/s3/) bin. Download the file into a buffer:

In [4]:
buf = urllib2.urlopen(url).read()

Output the contents of the file:

In [5]:
hexdump.hexdump(buf)

00000000: 64 65 66 20 45 28 69 6E  70 75 74 5F 62 79 74 65  def E(input_byte
00000010: 29 3A 0A 20 20 72 65 74  75 72 6E 20 63 68 72 28  ):.  return chr(
00000020: 69 6E 74 28 27 30 78 32  61 27 20 2B 20 69 6E 70  int('0x2a' + inp
00000030: 75 74 5F 62 79 74 65 2C  20 62 61 73 65 3D 31 36  ut_byte, base=16
00000040: 29 29 0A 0A E2 A9 96 E2  A8 B2 E2 A9 96 E2 A9 B3  ))..............
00000050: E2 A9 A2 E2 A9 83 E2 A9  82 E2 A9 AB E2 A9 A2 E2  ................
00000060: A8 B2 E2 A8 B5 E2 A9 AC  E2 A9 89 E2 A9 84 E2 A9  ................
00000070: AF E2 A9 B4 E2 A9 8B E2  A9 93 E2 A9 81 E2 A9 A8  ................
00000080: E2 A9 89 E2 A9 93 E2 A9  82 E2 A9 93 E2 A9 A4 E2  ................
00000090: A9 97 E2 A8 B4 E2 A9 A7  E2 A9 A4 E2 A9 87 E2 A9  ................
000000A0: A8 E2 A9 B0 E2 A9 A3 E2  A9 B9 E2 A9 82 E2 A9 A8  ................
000000B0: E2 A9 A3 E2 A9 B9 E2 A9  82 E2 A8 B4 E2 A9 8E E2  ................
000000C0: A9 AA E2 A9 91 E2 A9 A7  E2 A9 99 E2 A8 B2 E2 A8  ................

The content of the file shows what seems to be an encryption function `E()` followed by a byte buffer. The `E()` function takes each byte value from an unknown buffer and prepends the value `0x2a` to it, and finally converts it to a character. The appended buffer seems to be the output of this function.

Before the character conversion, the 16bit values can be seen as [Unicode Codepoints](https://en.wikipedia.org/wiki/Unicode#Code_point_planes_and_blocks) in the range [U+2A00 to U+2AFF (Supplemental Mathematical Operators](https://www.utf8-chartable.de/unicode-utf8-table.pl?start=10752&view=3)

The character conversion is done with [UTF-8](https://en.wikipedia.org/wiki/UTF-8) encoding, which gives the following result:

In [6]:
encrypted = buf[0x44:].decode('utf-8').replace('\n', '')
print(encrypted)

⩖⨲⩖⩳⩢⩃⩂⩫⩢⨲⨵⩬⩉⩄⩯⩴⩋⩓⩁⩨⩉⩓⩂⩓⩤⩗⨴⩧⩤⩇⩨⩰⩣⩹⩂⩨⩣⩹⩂⨴⩎⩪⩑⩧⩙⨲⨹⩫⩚⩓⩂⩨⩢⩭⩑⩧⩚⨲⩖⨰⩉⩈⩬⩶⩤⩘⩉⩧⩚⩭⩸⩨⩚⩹⩅⩧⩐⩃⨱⩺⩤⩇⩆⩹⩤⩃⨱⩨⩚⩮⩒⩬⩣⩩⨱⨰⩡⩇⩬⩺⩌⩗⨱⩨⩡⨲⩖⩹⩐⩫⩧⩸⩹⩕⩩⩂⨶⩤⩪⨯⨯⨯⨹⩉⩪⩑⩘⩶⨯⨯⨯⨯⩓⩌⩳⩧⩧⩇⩣⩪⩉⨫⩅⩷⩃⩅⩧⩸⩗⩃⩤⩉⩌⩦⩪⨯⨯⨯⨯⩩⨹⩎⩺⩉⨵⩳⩦⩔⩈⩳⨯⨳⩹⩆⩂⩮⩉⩹⩏⩧⩙⩕⩬⩷⨰⩪⩚⨱⩡⨹⩄⩩⩢⩗⩧⩌⩎⩕⩍⩤⩱⩢⩴⩡⩏⩌⨴⩶⩱⩈⩈⩂⩄⩫⩃⩲⨸⩪⩣⩤⩡⨫⨶⩈⩑⩭⩲⩎⩖⩵⩰⩲⨰⩐⩃⩫⩈⩏⩅⩢⩉⩑⨯⩂⩣⩣⩮⩰⩪⩓⩙⩩⨴⩧⩐⩤⩗⩭⩈⩒⩗⩗⩵⩯⩳⩸⩁⨲⩱⨸⩊⩢⩡⩹⩉⩸⩄⩯⩏⩧⩃⩇⩣⩪⩉⨶⩭⨱⩹⩆⩔⩶⩌⩹⩌⩺⩳⩑⨶⩄⩡⩊⩨⩚⩚⨶⩩⩨⩅⩅⩅⩨⩕⩉⩒⨯⩡⩸⨷⨵⩎⩭⩅⩌⩕⨶⩴⩲⨴⩏⩚⩆⩅⩕⩫⩶⩅⩵⩎⩎⩣⩣⩮⩰⩪⩓⩙⩩⨴⩴⩮⩑⩦⩤⩇⨫⩋⩹⩂⩶⩸⩔⩨⩎⩇⩖⩅⩓⨹⩘⩶⩦⩤⩉⩎⩧⩰⩃⨴⩩⨸⨴⩣⩏⩓⩡⩵⩍⩌⩸⨱⩮⩡⩮⩁⩕⩡⩙⩇⨳⩈⩗⩊⩱⩎⩉⩂⩯⩧⩢⩤⩩⩥⨶⩂⩯⩖⩮⩮⩡⩊⩮⩴⩩⩵⩈⩆⩓⩡⩁⩏⩌⩁⨲⩋⩺⩺⨫⩨⨴⩷⩔⨵⨵⩈⩡⩭⨷⩇⩳⩮⩊⩭⩎⩺⩣⩶⩈⩮⩐⨴⩙⩂⩮⩉⩹⩐⩦⩥⩉⩗⨱⩦⩭⩣⩪⩉⨹⨹⨸⩨⩡⩗⩫⩚⩩⩍⩪⩱⩑⩈⩂⩙⩔⩯⩩⩯⩈⩘⩭⩺⨹⨱⩯⩳⩡⨵⩩⩭⩒⩇⩆⩱⩮⩚⨯⩳⩬⩒⩇⩪⩖⩷⩯⩒⩏⨸⩊⩒⩧⩌⩂⩓⩗⩤⩖⨸⩫⩤⩆⩔⨴⩂⩘⩋⩅⩮⩺⩘⩑⩎⩙⩪⩅⩎⩮⩑⨹⨸⩖⩖⩫⨫⩅⩓⩮⩕⩧⩺⩑⩊⩑⩕⩉⩂⩘⩢⩗⩌⩶⩈⩹⩍⩪⨴⩔⩁⩉⩃⩧⨽⨽⨊


In the next step we define the following function `D()` needed to reconstruct the original buffer:

In [7]:
def D(input_char):
    code_point = ord(input_char)
#    print("%s: %s (%s)" % (input_char, input_char.encode('utf-8'), hex(code_point)))
    byte_value = code_point - 0x2a00
    return chr(byte_value)

The function `D()` applied iteratively to the ciphertext reveals the original buffer:

In [8]:
decrypted = ''.join([D(c) for c in encrypted])
print(decrypted)

V2VsbCBkb25lIDotKSAhISBSdW4gdGhpcyBhcyB4NjQgY29kZSBhbmQgZ2V0IHlvdXIgZmxhZyEgPC1zdGFydC1hZnRlci10aGlzLW1ha2VyPkgxyUiB6dj///9IjQXv////SLsggGcjI+EwCEgxWCdILfj////i9NzI5sfTHs/3yFBnIyOgYUlw0jZ1a9DibWgLNUMdqbtaOL4vqHHBDkCr8jcda+6HQmrNVupr0PCkHOEbIQ/BccnpjSYi4gPdWmHRWWuosxA2q8JbayIxDoOgCGcjI6m1yFTvLyLzsQ6DaJhZZ6ihEEEhUIR/ax75NmELU6tr4OZFEUkvEuNNccnpjSYi4tnQfdG+KyBvxThNGVES9XvfdINgpC4i84cOSauMLx1nanAUaYG3HWJqNIBogbdie6BoVnnaJntiuHFSaAOLA2Kzz+h4wT55Ham7GsnJmNzcvHnP4YBnIyPfeIW1fmcjI998haWkZiMjqQHBYToioHXmz91osa5imRGFqnZ/slRGjVwoRO8JRgLBSWdV8kdFT4BXKEnzXQNYjENnQ98VVk+ESnUgzQJQUIBXbWLvHyMj4TAICg==



This in turn seems to be yet another Base64 encoded buffer:

In [9]:
decoded = base64.b64decode(decrypted)
hexdump.hexdump(decoded)

00000000: 57 65 6C 6C 20 64 6F 6E  65 20 3A 2D 29 20 21 21  Well done :-) !!
00000010: 20 52 75 6E 20 74 68 69  73 20 61 73 20 78 36 34   Run this as x64
00000020: 20 63 6F 64 65 20 61 6E  64 20 67 65 74 20 79 6F   code and get yo
00000030: 75 72 20 66 6C 61 67 21  20 3C 2D 73 74 61 72 74  ur flag! <-start
00000040: 2D 61 66 74 65 72 2D 74  68 69 73 2D 6D 61 6B 65  -after-this-make
00000050: 72 3E 48 31 C9 48 81 E9  D8 FF FF FF 48 8D 05 EF  r>H1.H......H...
00000060: FF FF FF 48 BB 20 80 67  23 23 E1 30 08 48 31 58  ...H. .g##.0.H1X
00000070: 27 48 2D F8 FF FF FF E2  F4 DC C8 E6 C7 D3 1E CF  'H-.............
00000080: F7 C8 50 67 23 23 A0 61  49 70 D2 36 75 6B D0 E2  ..Pg##.aIp.6uk..
00000090: 6D 68 0B 35 43 1D A9 BB  5A 38 BE 2F A8 71 C1 0E  mh.5C...Z8./.q..
000000A0: 40 AB F2 37 1D 6B EE 87  42 6A CD 56 EA 6B D0 F0  @..7.k..Bj.V.k..
000000B0: A4 1C E1 1B 21 0F C1 71  C9 E9 8D 26 22 E2 03 DD  ....!..q...&"...
000000C0: 5A 61 D1 59 6B A8 B3 10  36 AB C2 5B 6B 22 31 0E  Za.Yk...6..[k"1.

The result is [x86-64](https://en.wikipedia.org/wiki/X86-64) machine code, which we can disassemble with the [Capstone Engine](http://www.capstone-engine.org/):

In [10]:
CODE = decoded[0x52:]
md = Cs(CS_ARCH_X86, CS_MODE_64)
for i in md.disasm(CODE, 0x0000):
    print("0x%04x: %s %s\t%s" %(i.address, hexdump.dump(i.bytes).ljust(32), i.mnemonic, i.op_str))


0x0000: 48 31 C9                         xor	rcx, rcx
0x0003: 48 81 E9 D8 FF FF FF             sub	rcx, -0x28
0x000a: 48 8D 05 EF FF FF FF             lea	rax, [rip - 0x11]
0x0011: 48 BB 20 80 67 23 23 E1 30 08    movabs	rbx, 0x830e12323678020
0x001b: 48 31 58 27                      xor	qword ptr [rax + 0x27], rbx
0x001f: 48 2D F8 FF FF FF                sub	rax, -8
0x0025: E2 F4                            loop	0x1b
0x0027: DC C8                            fmul	st(0), st(0)
0x0029: E6 C7                            out	0xc7, al
0x002b: D3 1E                            rcr	dword ptr [rsi], cl
0x002d: CF                               iretd	
0x002e: F7 C8 50 67 23 23                test	eax, 0x23236750
0x0034: A0 61 49 70 D2 36 75 6B D0       movabs	al, byte ptr [0xd06b7536d2704961]
0x003d: E2 6D                            loop	0xac
0x003f: 68 0B 35 43 1D                   push	0x1d43350b
0x0044: A9 BB 5A 38 BE                   test	eax, 0xbe385abb


Now let's do a symbolic execution of the code using the [Unicorn Engine](https://www.unicorn-engine.org/docs/tutorial.html):

In [11]:
mu = Uc(UC_ARCH_X86, UC_MODE_64)
mu.mem_map(0, 2*1024*1024)
mu.mem_write(0, CODE)
try:
    mu.emu_start(0, 512)
except UcError as e:
    pass
mem = mu.mem_read(0, 512)
hexdump.hexdump(str(mem))

00000000: 48 31 C9 48 81 E9 D8 FF  FF FF 48 8D 05 EF FF FF  H1.H......H.....
00000010: FF 48 BB 20 80 67 23 23  E1 30 08 48 31 58 27 48  .H. .g##.0.H1X'H
00000020: 2D F8 FF FF FF E2 F4 FC  48 81 E4 F0 FF FF FF E8  -.......H.......
00000030: D0 00 00 00 41 51 41 50  52 51 56 48 31 D2 65 48  ....AQAPRQVH1.eH
00000040: 8B 52 60 3E 48 8B 52 18  3E 48 8B 52 20 3E 48 8B  .R`>H.R.>H.R >H.
00000050: 72 50 3E 48 0F B7 4A 4A  4D 31 C9 48 31 C0 AC 3C  rP>H..JJM1.H1..<
00000060: 61 7C 02 2C 20 41 C1 C9  0D 41 01 C1 E2 ED 52 41  a|., A...A....RA
00000070: 51 3E 48 8B 52 20 3E 8B  42 3C 48 01 D0 3E 8B 80  Q>H.R >.B<H..>..
00000080: 88 00 00 00 48 85 C0 74  6F 48 01 D0 50 3E 8B 48  ....H..toH..P>.H
00000090: 18 3E 44 8B 40 20 49 01  D0 E3 5C 48 FF C9 3E 41  .>D.@ I...\H..>A
000000A0: 8B 34 88 48 01 D6 4D 31  C9 48 31 C0 AC 41 C1 C9  .4.H..M1.H1..A..
000000B0: 0D 41 01 C1 38 E0 75 F1  3E 4C 03 4C 24 08 45 39  .A..8.u.>L.L$.E9
000000C0: D1 75 D6 58 3E 44 8B 40  24 49 01 D0 66 3E 41 8B  .u.X>D.@$I..f>A.

The code writes the flag to memory near offset 0x00000140. It says: `{msoc_rulez}`. What I can say is: this challenge rules ! :-)